In [ ]:
---
title: Twitter Following Alpha
author: Yifan
description: getting the twitter following data 
show-code: False
--- 

In [16]:
import os
import boto3
from boto3 import session
import io
#from io import StringIO # python3; python2: BytesIO 

import mercury as mr
import pandas as pd
import numpy as np

from botocore import config


#print(secret_key)


In [6]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('max_colwidth',100)

In [ ]:
proxies = {
        'https': 'http://127.0.0.1:7890',  
        'http': 'http://127.0.0.1:7890'  
    }


In [20]:
#session = boto3.session.Session(aws_access_key_id=key_id, aws_secret_access_key=secret_key)

s3_client = boto3.client('s3')


In [ ]:
import mercury as mr
 
# add select widget

choices = ["a16z_common","chinese_common","coinbase_common","dragonfly_common","HackVC_common","HaunVentures_common","KOL_common","paradigm_common",\
          "polychain_common","robot_common","variant_common","Final"]

group_selection = mr.Select(value="dragonfly_common", choices=choices, label="选择VC组")


import mercury as mr
 
# add range widget
your_range = mr.Range(value=[2023, 2023], min=2020, max=2023, label="请选择你的想要的数据时间范围", step=1)

#print(group_selection.value)

In [ ]:
#获取想要的列
"""
目前选择 name, username,can_dm, created_at,bio,followers_count,following_count,profile_url

"""
def clean_columns(df):

    columns = ['name', 'username','can_dm', 'created_at',\
               'bio','followers_count','following_count','profile_url']
    
    df_clean = df[columns]
    
    
    return df_clean
    




# 每个KOL关注的新创立的账号

# 有些时间再DM列，start 是反的 -- 重新爬

def get_by_time(df,start=2022,end=2023,get_time=True):
    
    
    if get_time == True:
        
        count = 1
        
        larger_start = df['created_at'].apply(lambda x: x.split(',')[0]).str.endswith(str(start))
            
        df_list = df[larger_start]
        
        while start + count <= end:
            
            time = start + count
            
            smaller_end = df['created_at'].apply(lambda x: x.split(',')[0]).str.endswith(str(time))
            
            df_temp = df[smaller_end]
            
            df_list = pd.concat([df_list,df_temp])
            
            count += 1

        
        #df_time = df_list.sort_values('created_at',ascending=False)
        
        return df_list
    
    else:
        print("set get_time = true")
        return 1
        


In [ ]:
data_bucket_name = "twiiterfollowing"
file_path = group_selection.value + ".csv"

response = s3_client.get_object(Bucket=data_bucket_name,Key=file_path)
response_body = response['Body'].read()

df = pd.read_csv(io.BytesIO(response_body))


In [ ]:
# 时间调整
df_time = get_by_time(df,start=your_range.value[0],end=your_range.value[1])

In [ ]:
# 排序方法

sort_list = ["按照时间排序","按照VC共同follow的数量排序","按照项目本身follower的数量排序"]
sort_as = mr.Select(value=sort_list[2], choices=sort_list, label="选择排序方式")


if sort_as.value == sort_list[0]:
    df_final = df_time.sort_values(by='created_at')
elif sort_as.value == sort_list[1]:
    df_final = df_time.sort_values(by='size')
elif sort_as.value == sort_list[2]:
    df_final = df_time.sort_values(by='followers_count')
    



In [ ]:
print("1. size 代表所有这些VC研究员共同follow的数量")


In [ ]:
print("2. username 代表twitter账号，可以直接搜索")

In [ ]:
print("3. 不同的group代表不同的VC机构研究员，每个VC group 包含旗下所有研究员数据")


In [ ]:
print("4. Final 代表所有的VC数据汇总")

In [ ]:
print("5. 乱码代码是各种表情符号，直接忽略就行，看英文部分")

In [ ]:
df_end = df_final.reset_index(drop=True)
df_end

In [ ]:
## 加入写入逻辑

"""s3 = boto3.resource(
    's3',
    region_name='ap-southeast-2',
    aws_access_key_id=key_id,
    aws_secret_access_key=secret_key
)

#s3.Bucket(data_bucket_name).upload_file(r'\\' + file_path, file_path)


s3_client.upload_file('/' + file_path, data_bucket_name, file_path)




bucket = data_bucket_name # already created on S3
csv_buffer = StringIO()
df_end.to_csv(csv_buffer)
#s3_resource = boto3.resource('s3')
s3.Object(bucket, file_path).put(Body=csv_buffer.getvalue())"""
